<a href="https://colab.research.google.com/github/damilojohn/Playlist_Generator/blob/main/Copy_of_Generating_Afrobeats_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing necessary libraries
!pip install sentence-transformers
!pip install wandb

In [2]:
#songs csv 
!wget https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/spotify_transformed.csv

--2023-02-02 00:54:02--  https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/spotify_transformed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167653 (164K) [text/plain]
Saving to: ‘spotify_transformed.csv’

spotify_transformed 100%[===================>] 163.72K  --.-KB/s    in 0.009s  

2023-02-02 00:54:03 (17.5 MB/s) - ‘spotify_transformed.csv’ saved [167653/167653]



In [ ]:
#setting up wandb
!wandb login
import wandb

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
config = {
    'Name' : 'Embeddings for afrobeats lyrics',
    'Dataset' : 'spotify api lyrics',
    'Model' : 'msmarco-miniLM-pretrained',
    'notes':'No text cleaning was performed yet.creating a baseline performance'
}
wandb.init(project='Playlist Generator',entity='damilojohn',config=config)

In [3]:
from sentence_transformers import SentenceTransformer,util
from tqdm.auto import tqdm
import pandas as pd 
import transformers
import pickle 
from huggingface_hub import hf_hub_download
import re

In [4]:
#loading song lyrics into a dataset object 
data_path = '/content/spotify_transformed.csv'

In [5]:
embedder = SentenceTransformer('msmarco-distilbert-base-v4')

In [6]:
songs= pd.read_csv(data_path)

In [7]:
def clean_verses(lyrics_uncleaned):
  ''' takes a list of verses as input and removes useless characters such as /n and stuff '''
  lyrics_cleaned = lyrics_uncleaned.split('\n')
  lyrics_cleaned = [i for i in lyrics_cleaned if i]
  for i,lyrics in enumerate(lyrics_cleaned) :
    if not lyrics[0].isalpha():
      del lyrics_cleaned[i]
  return lyrics_cleaned 


In [37]:
embeddings = embedder.encode(new_verses,)
assert len(new_verses) == len(embeddings) , 'embeddings and lyrics are mismatched '

In [45]:
#trying out semantic search 
query = ['floating through all my troubles']
query_embedding = embedder.encode(query)
hits = util.semantic_search(query_embedding,embeddings,top_k = 6)
hits

[[{'corpus_id': 1634, 'score': 0.45986294746398926},
  {'corpus_id': 2920, 'score': 0.40663743019104004},
  {'corpus_id': 2896, 'score': 0.40663743019104004},
  {'corpus_id': 424, 'score': 0.3973277509212494},
  {'corpus_id': 99, 'score': 0.3973277509212494},
  {'corpus_id': 1585, 'score': 0.37971732020378113}]]

In [46]:
for hit in hits[0]:
  print(new_dict[new_verses[hit['corpus_id']]])

Last Last
Free Mind
Free Mind
Essence (feat. Justin Bieber & Tems)
Essence (feat. Justin Bieber & Tems)
For My Hand (feat. Ed Sheeran)


In [30]:
sample = sample[(sample['for_lyrics'].str.len() != 0 )]

In [9]:
sample['for_lyrics'] = sample['for_lyrics'].apply(clean_verses,)

In [32]:
dictt=dict(zip(sample.song_name,sample.for_lyrics))

In [ ]:
verses = []
for i in sample['for_lyrics'].values:
  verses.append(i)
verses = [i for i in verses if i] 
new_verses = [] 
for i in verses :
  for j in i :
    new_verses.append(j)
new_verses

In [36]:
new_dict={}
for i,j in dictt.items():
    for k in j:
        new_dict[k]=i

In [51]:
mappings = pd.DataFrame.from_dict(new_dict,orient='index')

In [57]:
mappings.to_csv('/content/song_mappings.csv')

In [47]:
import pickle
path = '/content/em.pkl'
with open(path,'wb') as pk:
  pickle.dump(embeddings,pk)

In [48]:
from huggingface_hub import login 
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [49]:
from huggingface_hub import hf_hub_download

In [ ]:
text = 'Call me at 415-555-1011 tomorrow. 415-555-9999 is my office.'

In [ ]:
reg = re.compile(r'\d{3}-\d{3}-\d{4}')
finds = reg.search(text)
finds.group()

'415-555-1011'

In [ ]:
heroregex = re.compile('batman|Tina Fey')
match1 = heroregex.search(' the batman had Tina Fey in it ')
match1.group()

'batman'